![header](http://eurogoos.eu/wp-content/uploads/SOCIB-logo.jpg)

# SOCIB API TRAINING
<div style="text-align: right"><i> 03-Part-one-out-of-04 </i></div>

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item">
    <li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span>
    </li>
    <li><span><a href="#Setup" data-toc-modified-id="Setup-2">
        <span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span>
        <ul class="toc-item">
            <li>
                <span><a href="#Importing-modules" data-toc-modified-id="Importing-modules-2.1">
                    <span class="toc-item-num">2.1&nbsp;&nbsp;</span>Importing modules</a></span>
            </li>
            <li>
                <span><a href="#API-token" data-toc-modified-id="API-token-2.2">
                    <span class="toc-item-num">2.2&nbsp;&nbsp;</span>API token</a></span>
            </li>
        </ul>
    <li>
        <span><a href="#Available-services-for-SOCIB-entries/files" data-toc-modified-id="Available-services-for-SOCIB-entries/files"><span class="toc-item-num">3&nbsp;&nbsp;</span>Available services for SOCIB entries/files:</a></span>
        <ul>
            <li><span><a href="#Full-list-of-services" data-toc-modified-id="Full-list-of-services"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Full list of services</a></span></li>
            <li><span><a href="#Services-overview" data-toc-modified-id="Services-overview"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Services overview</a></span></li>
                <ul>
                    <li><span><a href="#Dapp" data-toc-modified-id="Dapp"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Dapp</a></span></li>
                    <li><span><a href="#Jwebchart" data-toc-modified-id="Jwebchart"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Jwebchart</a></span></li>
                    <li><span><a href="#lw4nc2" data-toc-modified-id="lw4nc2"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>lw4nc2</a></span></li>
                    <li><span><a href="#Profiler-viewer" data-toc-modified-id="Profiler-viewer"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Profiler viewer</a></span></li>
                </ul>
        </ul>
     </li>
    </ul>
</div>

---

# ENTRIES

## Introduction 

SOCIB API is a door users can knock-on in order to get information about the Balearic Islands Coastal Ocean Observing and Forecast System (SOCIB). SOCIB API is represented by an generic url (SOCIB API url). The elements that trigger a response when added to the generic API url are called `ENDPOINTS`.
Among the present posibilities:
<ul>
    <li>measured variables (<span class="alert-info">/standard-variables/</span>)</li>
    <li>stock of instruments (<span class="alert-info">/instrument-types/</span>) and platforms(<span class="alert-info">/platform-types/</span>), 
</li>
    <li>data maturity (<span class="alert-info">/processing-levels/</span> and <span class="alert-info">/data-modes/</span>)</li>
    <li>kind of data (<span class="alert-info">/feature-types/</span>)</li>
    <li>data entities (<span class="alert-success"><b>/entries/<b></span>, <span class="alert-info">/data-sources/</span>, <span class="alert-info">/platforms/</span>, <span class="alert-info">/data-products/</span>)
    </li>

</ul>

<br>This notebook will focus then on the <span class="alert-success" style=""><b>/entries/</b></span> endpoint.

---

## Setup

### Importing modules

We will relly on a set of python modules to deal with SOCIB API next.<br> `Please run the next cell` so that they can be used by the present Jupyter Notebook:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import folium
from folium import plugins
from owslib.wms import WebMapService
import xmltodict
import os
import urllib
import xarray
import json
import requests
import IPython
%matplotlib inline

<div class="alert alert-block alert-warning" style="margin-left: 2em">
<b>Tip!</b>
    
***  
If any of them raises any error please run prior to load it and in a dedicated cell, either:<ul><li> <i>`!conda install packageName --yes`</i></li> or <li><i>`!pip install packageName --yes`</i></li></ul>

### API token

To be able to query SOCIB API you will need first a <i>token</i> (api key).<br>To get one please visit the [API home page](http://api.socib.es/home/) and fill-in the form at bottom. An email will be sent to you with such <i>token</i>.

`Please run the next cell if you wanna have a glimpse to API home page`:

In [ ]:
IPython.display.HTML('<iframe src="http://api.socib.es/home/" width=100% height=500></iframe>')

`Please set in the next cell your api_key and run the cell below to load it in memory for later use`:

In [ ]:
api_key = '' #write here the token emailed to you

In [ ]:
api_url = 'http://api.socib.es'
headers = {
    'accept': 'application/vnd.socib+json',
    'apikey': api_key,
}

<div class="alert alert-block alert-warning" style="margin-left: 2em">
<b>Tip!</b>
    
***  
If you do not remember your token or wanna ask for a new one please reach <i>data.center@socib.es</i> with the following `subject`: 'SOCIB API TOKEN: UPDATE/REMIND REQUEST'

---

## Available services for SOCIB entries/files

### Full list of services

In order to ease data exploration there are a number of services available for remote-access or download the different entries/files produced by SOCIB. There are potentially 4 services in total and the full list can be obtained also by querying SOCIB API as follows:

In [ ]:
end_point = '/entries-data-services/'
url = '%s%s' % (api_url, end_point)
viewers_request = requests.get(url, headers=headers)
viewers_response = json.loads(viewers_request.text)
print('API query url for viewers: '+ url)
pandas.DataFrame.from_dict(viewers_response)

Nevertheless, not all 4 are always available for a given entry/file. Typically the first 3 services are guaranteed for every entry/file whereas the fourth applies only in case of gridded data. In the next section we will see the different cases:

### Services overview

#### Http_file

This service enables the download of files. It is the basic service provided for entries/files produced and it is basically an url that via HTTP request trigger the download of the file. Try it out!

Supose we wanna download data time serie data:

In [ ]:
end_point = '/entries/?feature_type=timeSeries'
url = '%s%s' % (api_url, end_point)
entries_request = requests.get(url, headers=headers)
entries_response = json.loads(entries_request.text)
print('API query url: '+ url)
print('Http_file service url for the first entry/file on list: ' + entries_response['results'][0]['services']['http_file']['url'])

Click on the above url to perform the download manually or just run the code below for doing it programatically:

In [ ]:
out_put_dir = os.getcwd() #Set the autput dir path. By default: current working directory

In [ ]:
entry_http_file_url =  entries_response['results'][0]['services']['http_file']['url']
filename = entry_http_file_url.split('/')[-1]
filePath = os.path.join(out_put_dir,filename)
urllib.request.urlretrieve(entry_http_file_url, filePath)

<div class="alert alert-block alert-warning" style="margin-left: 4em">
<b>Tip!</b>
    
***  
Check the output directory to see the file downloaded.

#### opendap

Opendap service allows users to access/explore files hosted in remore servers.
Let's see how to use it!

Querying for time series....

In [ ]:
end_point = '/entries/?feature_type=timeSeries'
url = '%s%s' % (api_url, end_point)
entries_request = requests.get(url, headers=headers)
entries_response = json.loads(entries_request.text)
print('API query url: '+ url)
print('opendap service url for the first entry/file on list: ' + entries_response['results'][0]['services']['opendap']['url'])

Let's access the above file trough the opendap-url

In [ ]:
entry_opendap_url =  entries_response['results'][0]['services']['opendap']['url']
ds = xarray.open_dataset(entry_opendap_url)
ds.close()
ds

Let's make also a quick plot of one of its variables:

In [ ]:
ds['AIR_PRE'].plot()

<div class="alert alert-block alert-warning" style="margin-left: 4em">
<b>Tip!</b>
    
***
Check [XARRAY documentation](http://xarray.pydata.org/en/stable/) for testing how to deal with netCDF files.

#### thredds_catalog

Thredds_catalog service is actually pointing toward the file dedicated url in SOCIB Thredds Web Catalog for a troughout checking. Let's see how it looks!

Again, for example querying for timeSerie data....

In [ ]:
end_point = '/entries/?feature_type=timeSeries'
url = '%s%s' % (api_url, end_point)
entries_request = requests.get(url, headers=headers)
entries_response = json.loads(entries_request.text)
print('API query url: '+ url)
print('thredds catalog service url for the first entry/file on list: ' + entries_response['results'][0]['services']['thredds_catalog']['url'])

Let's check it above page in a iframe:

In [ ]:
entry_thredds_catalog_url =  entries_response['results'][0]['services']['thredds_catalog']['url']
IPython.display.HTML('<iframe src="'+entry_thredds_catalog_url+'" width=100% height=500></iframe>')

#### WMS

WMS services are only avaialable for gridded like data. And enables retrieveing discrete information from such grid as well as map fields. Let's check it!

Querying for grided like data....

In [ ]:
end_point = '/entries/?feature_type=grid'
url = '%s%s' % (api_url, end_point)
entries_request = requests.get(url, headers=headers)
entries_response = json.loads(entries_request.text)
print('API query url: '+ url)
print('WMS service url for the first entry/file on list: ' + entries_response['results'][0]['services']['wms']['url'])

<ul><li><b>Map info</b></li></ul>

In [ ]:
entry_wms_url = entries_response['results'][0]['services']['wms']['url']
wms = WebMapService(entry_wms_url)
for layer in wms.contents.keys():
    print(layer + ' = ' + wms[layer].title)

Let's choose one of the above variables and calculate the center of its bounding box: run the next cells

In [ ]:
target = 'WSPE_ORG'
lon = (wms[target].boundingBox[0] + wms[target].boundingBox[2]) / 2.
lat = (wms[target].boundingBox[1] + wms[target].boundingBox[3]) / 2.
center = lat, lon
print(center)

Let's now display the chosen variable on a map centered in the abobe lat,lon point: run the next cells

<div class="alert alert-block alert-warning" style="margin-left: 4em">
<b>Warning!</b>
    
***
Be aware that internet explorer browsers won't render the below map. If you are using such browser for running the notebook please change to another (i.e chrome!)

In [ ]:
#Style definition: first style available - try others by changing the value of the index
style_index = 0
style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][style_index] 
print(style)

In [ ]:
#legend definition: first legend available - try others by changing the value of the index
legend_index = 0
legend = [wms[target].styles[key]['legend'] for key in wms[target].styles.keys()][legend_index]
print(legend)

In [ ]:
ini = wms[target].timepositions[0].strip()
end = wms[target].timepositions[-1].strip()
time_coverage = (ini+'/'+end).split('/P')[0]
print(time_coverage)

In [ ]:
#Layer specification
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
)
time_layer = plugins.TimestampedWmsTileLayers(w,period='P1H',time_interval=time_coverage)

In [ ]:
#layer display on map
m = folium.Map(
    location=center,
    zoom_start=2,
    control_scale=True
)
time_layer.add_to(m)
#layer style
legend_html = '<div style="position: fixed; bottom: 50px; right:0px; z-index:9999"><img src="'+legend+'"></div>'
#add legend to map
m.get_root().html.add_child(folium.Element(legend_html))
#show map
m

<ul><li><b>Discrete info</b></li></ul>

Let's choose a location: i.e the center of the layer

In [ ]:
target = 'WSPE_ORG'
lon = (wms[target].boundingBox[0] + wms[target].boundingBox[2]) / 2.
lat = (wms[target].boundingBox[1] + wms[target].boundingBox[3]) / 2.
center = lat, lon
print(center)

Let's choose a time range: i.e the whole time covered by the file

In [ ]:
ini = wms[target].timepositions[0].strip()
end = wms[target].timepositions[-1].strip()
time_coverage = (ini+'/'+end).split('/P')[0]
print(time_coverage)

Let's get the rectangle whith such point as centroid:

In [ ]:
#Remember, the grid_size of the product is 0.5
lon_min = 0.8
lon_max = 1
lat_min = 38.6
lat_max = 38.9
bbox = ','.join([str(lon_min), str(lat_min), str(lon_max), str(lat_max)])

Let's build the query to the WMS now:

In [ ]:
query = entry_wms_url+'?'+\
'SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0'+\
'&TIME='+time_coverage+\
'&LAYERS='+target+\
'&QUERY_LAYERS='+target+\
'&BBOX='+bbox
print(query)

Let's make the query!:

In [ ]:
response = requests.get(query)
data = xmltodict.parse(response.content)
print('Time serie at point: '+','.join([str(lat),str(lon)]))
for info in data['FeatureInfoResponse']['FeatureInfo']:
    print(info['time'])
    print(info['value'])
    print('------------')

---

## Next tutorial

<div class="alert alert-block alert-success" style="margin-left: 2em">
<b>More!</b>
    
***  
To see way more about SOCIB entries next dedicated notebooks:
<ul>
    <li><span><a href="../entries/04-entry-data-as-json.ipynb">04-entry-data-as-json</a></span></li>
</ul>